In [1]:
from fastai.text import *
import numpy as np
import pickle
import sentencepiece as spm
from tqdm import tqdm

In [2]:
!pwd

/home/hap-101/Downloads/code-mixed/code-mixed/language_model


In [3]:
src = 'data/trn_tokens'
with open(f'{src}/0_trainaa.txt.pkl', 'rb') as f:
    tokens = pickle.load(f)

In [4]:
tokens

['▁',
 'xxmaj',
 '▁notification',
 '▁of',
 '▁',
 'xxmaj',
 '▁national',
 '▁',
 'xxmaj',
 '▁eligibility',
 '▁and',
 '▁',
 'xxmaj',
 '▁entrance',
 '▁',
 'xxmaj',
 '▁test',
 '▁',
 'xxup',
 '▁(',
 'neet',
 '-2018',
 ')',
 '▁to',
 '▁be',
 '▁conducted',
 '▁for',
 '▁admission',
 '▁in',
 '▁',
 'xxup',
 '▁mbbs',
 '▁and',
 '▁',
 'xxup',
 '▁bds',
 '▁has',
 '▁been',
 '▁released',
 '.',
 '▁pariksha',
 '▁6',
 '▁may',
 '▁(',
 'ravivaar',
 ')',
 '▁ko',
 '▁yojit',
 '▁kii',
 '▁jaaegi',
 '.',
 '▁',
 'xxup',
 '▁neet',
 '▁exam',
 '▁is',
 '▁considered',
 '▁to',
 '▁be',
 '▁one',
 '▁of',
 '▁the',
 '▁toughest',
 '▁exams',
 '▁in',
 '▁the',
 '▁country',
 '.',
 '▁agar',
 '▁aap',
 '▁chaahate',
 '▁hai',
 '▁entrens',
 '▁pariksha',
 '▁main',
 '▁kisi',
 '▁bhi',
 '▁tarah',
 '▁kii',
 '▁pareshaani',
 '▁na',
 '▁ho',
 '.',
 '▁',
 'xxmaj',
 '▁for',
 '▁this',
 ',',
 '▁you',
 '▁must',
 '▁know',
 '▁these',
 '▁12',
 '▁important',
 '▁things',
 '▁first',
 '.',
 '▁-',
 '▁neet',
 '▁kii',
 '▁pravesh',
 '▁pariksha',
 '▁ke',
 '▁liye',

In [5]:
from inltk.tokenizer import HinglishTokenizer, handle_all_caps, handle_upper_case_first_letter

In [6]:
sp = spm.SentencePieceProcessor()
sp.Load(f"../tokenizer/hinglish_spm.model")
itos = [sp.IdToPiece(int(i)) for i in range(60000)]

In [7]:
# 60,000 is the vocab size that we chose in sentencepiece
hinglish_vocab = Vocab(itos)

In [8]:
tokenizer = Tokenizer(lang='hien', tok_func=HinglishTokenizer)

In [9]:
tokenizer.pre_rules.append(handle_all_caps)
tokenizer.pre_rules.append(handle_upper_case_first_letter)

In [10]:
def get_text_from_tokens(tokens):
    text = sp.decode_pieces(tokens)
    spc_tokens = text.split()
    for i, tk in enumerate(spc_tokens):
        if tk == 'xxmaj':
            spc_tokens[i+1] = spc_tokens[i+1][0].upper() + spc_tokens[i+1][1:]
        elif tk == 'xxup':
            spc_tokens[i+1] = spc_tokens[i+1].upper()
    ntext = ' '.join(spc_tokens)
    ntext = re.sub('xxmaj', '', ntext)
    ntext = re.sub('xxup', '', ntext)
    ntext = re.sub('▁', '', ntext)
    ntext = re.sub('  ', ' ', ntext)
    return ntext

In [11]:
get_text_from_tokens(tokens)

' Notification of National Eligibility and Entrance Test (NEET-2018) to be conducted for admission in MBBS and BDS has been released. pariksha 6 may (ravivaar) ko yojit kii jaaegi. NEET exam is considered to be one of the toughest exams in the country. agar aap chaahate hai entrens pariksha main kisi bhi tarah kii pareshaani na ho. For this, you must know these 12 important things first. - neet kii pravesh pariksha ke liye ummidavaar online registration 9 march 2018 tak kar sakate hai. NEET-2018 exam notification will be released, exam will be held on May 6 - Candidates can submit exam fees from March 8 to March 10, 2018. - online registration kii fees general category ke liye 1400 rupee he, vahiin SC or ST category ke liye 750 rupee fees tay kii gai he. - These NEET exams are conducted in MBBS / BDS in all medical institutes of the country except select institutes like AIIMS and JIPMER Puducherry. - jo chaatr NRI he. He will be required to provide passport number to register. NEET 201

In [16]:
base_dir = Path('/home/hap-101/Downloads/code-mixed/code-mixed')

src_dir = base_dir / 'language_model' / 'data' / 'val_tokens'
files = src_dir.ls()
for file in tqdm(files):
    gc.collect()
    with open(file, 'rb') as f:
        tokens = pickle.load(f)
    text = get_text_from_tokens(tokens)
    text = re.sub(r'\.', '\n', text)
    filename = str(file).split('/')[-1][:-4]
    with open(base_dir/f'data/hinglish/valid/{filename}', 'w') as f:
        f.write(text)
#     print(filename)
#     break

100%|██████████| 3157/3157 [06:08<00:00,  8.60it/s]


In [17]:
len(files)

3157

In [29]:
src_dir = base_dir / 'data' / 'hinglish' / 'train'
files = src_dir.ls()
filenames = [str(f).split('/')[-1] for f in files]

In [36]:
filenames[0]

'1_trainzblg.txt'

In [37]:
'0_trainzblg.txt' in filenames

True

In [42]:
org_fnames = []
for fname in filenames:
    ofname = fname[2:]
    assert '0_' + ofname in filenames and '1_' + ofname in filenames
    org_fnames.append(ofname)
org_fnames = list(set(org_fnames))
print(len(org_fnames))

2099


In [43]:
org_fnames[0]

'trainzbmo.txt'

In [45]:
with open(src_dir / '0_trainzbmo.txt', 'r') as f:
    l0 = f.readlines()
with open(src_dir / '1_trainzbmo.txt', 'r') as f:
    l1 = f.readlines()

In [46]:
len(l0), len(l1)

(5786, 5765)